In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict


In [2]:
import pyts
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [4]:
X_train = np.load("RavdessAudioOnlyNumpy__X_train_decimated_scaled_max.npy")
X_test = np.load("RavdessAudioOnlyNumpy__X_test_decimated_scaled_max.npy")
y_train = pd.read_csv("RavdessAudioOnlyNumpy__Y_train.csv")
y_test = pd.read_csv("RavdessAudioOnlyNumpy__Y_test.csv")

In [5]:
y_train = y_train["emotion"]
y_test = y_test["emotion"]

## KNN

### pyts

In [6]:
import pyts as pyts
from pyts import classification

ImportError: Numba needs NumPy 1.21 or less

In [7]:
clf = pyts.classification.KNeighborsClassifier(metric="euclidean")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

AttributeError: module 'pyts' has no attribute 'classification'

In [7]:
clf = pyts.classification.KNeighborsClassifier(metric="dtw")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.22435897435897437
F1-score [0.27027027 0.34080717 0.23809524 0.13496933 0.23350254 0.15789474
 0.17560976 0.15584416]
              precision    recall  f1-score   support

       angry       0.28      0.26      0.27        96
        calm       0.30      0.40      0.34        96
     disgust       0.28      0.21      0.24        48
     fearful       0.16      0.11      0.13        96
       happy       0.23      0.24      0.23        96
     neutral       0.14      0.19      0.16        48
         sad       0.17      0.19      0.18        96
   surprised       0.21      0.12      0.16        48

    accuracy                           0.22       624
   macro avg       0.22      0.21      0.21       624
weighted avg       0.22      0.22      0.22       624



## Shapelets Classifier

In [17]:
pip install tslearn.shapelets

ERROR: Could not find a version that satisfies the requirement tslearn.shapelets (from versions: none)
ERROR: No matching distribution found for tslearn.shapelets
Note: you may need to restart the kernel to use updated packages.


In [15]:
from tslearn.shapelets import ShapeletModel
from tslearn.shapelets import grabocka_params_to_shapelet_size_dict

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
n_ts, ts_sz = X_train.shape
n_classes = len(set(y_train))

# Set the number of shapelets per size as done in the original paper
shapelet_sizes = grabocka_params_to_shapelet_size_dict(n_ts=n_ts,
                                                       ts_sz=ts_sz,
                                                       n_classes=n_classes,
                                                       l=0.1,
                                                       r=1)

print('n_ts', n_ts)
print('ts_sz', ts_sz)
print('n_classes', n_classes)
print('shapelet_sizes', shapelet_sizes)

In [ ]:
shp_clf = ShapeletModel(n_shapelets_per_size=shapelet_sizes, optimizer="sgd", weight_regularizer=.01, max_iter=200, verbose=1)

In [ ]:
shp_clf.fit(X_train, y_train)

In [ ]:
'''
y_pred = shp_clf.predict(X_test)
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))
'''

In [ ]:
X_train_sh = shp_clf.transform(X_train)
X_test_sh = shp_clf.transform(X_test)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train_sh, y_train)


y_pred = clf.predict(X_test_sh)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

In [ ]:
clf = DecisionTreeClassifier(max_depth=8, random_state=42, class_weight="balanced")
clf.fit(X_train_sh, y_train)

y_pred = clf.predict(X_test_sh)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

## CNN

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling1D
from keras.layers import Conv1D, Activation, BatchNormalization

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
def build_simple_cnn(n_timesteps, n_outputs):
    model = Sequential()
    
    model.add(Conv1D(filters=3, kernel_size=3, activation='relu', input_shape=(n_timesteps, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.1))
    
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_train_cnn, y_train, test_size=0.2, stratify=y_train)

n_timesteps, n_outputs, n_features = X_train_cnn.shape[1], len(np.unique(y_train_cnn)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

In [ ]:
to_int_map={"angry":0, "calm":1, "disgust":2, "fearful":3, "happy":4, "neutral":5, "sad":6, "surprised":7}
y_train_cnn=np.asarray([to_int_map.get(string,0) for string in y_train_cnn])
y_test_cnn=np.asarray([to_int_map.get(string,0) for string in y_test])
y_val_cnn=np.asarray([to_int_map.get(string,0) for string in y_val_cnn])
y_test_cnn=np.asarray([to_int_map.get(string,0) for string in y_test])

In [ ]:
cnn = build_simple_cnn(n_timesteps, n_outputs)
cnn.summary()

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.0001)
#mc = ModelCheckpoint('best_model_cnn.h5', monitor='val_loss', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=8, verbose=1, restore_best_weights=True)

callbacks = [rlr, es] 
             #, mc]
batch_size = 16
mini_batch_size = int(min(X_train_cnn.shape[0]/10, batch_size))


history = cnn.fit(X_train_cnn, y_train_cnn, epochs=100, batch_size=mini_batch_size, callbacks=callbacks,
                      validation_data=(X_val_cnn, y_val_cnn))

In [ ]:
train_loss = history.history["loss"]
val_loss = history.history["val_loss"]

In [ ]:
plt.plot(train_loss, label="train")
plt.plot(val_loss, label="val")
plt.legend()
plt.show()

In [ ]:
y_test

In [ ]:
y_pred = np.argmax(cnn.predict(X_test_cnn), axis=1)

print('Accuracy %s' % accuracy_score(y_test_cnn, y_pred))
print('F1-score %s' % f1_score(y_test_cnn, y_pred, average=None))
print(classification_report(y_test_cnn, y_pred))